In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [77]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

from pandas.tseries.offsets import MonthEnd

In [78]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200818


In [79]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [80]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_Email")

#input
input_new = "EMAIL_CUST"
input_all = "EMAIL_CUST_ALL"

input_cust = "TOTAL_CUST"
input_cust_all = "TOTAL_CUST_ALL"

#out_put
output = "EMAIL_CUST_ALL"
output2 = "TOTAL_CUST_ALL"


#output(result)
result = "EmailOverview"
result_cha = "Email新規登録数_by_channel"
result_card = "Email新規登録数_by_card"
result_age = "Email新規登録数_by_age"

①Email登録データの読込＆集計

In [86]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
df = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
#文字化けするので修正
df['age'] = df['age'].str.replace('?','-')

#df2 = pd.read_csv(data_dir /f"{input_new}.csv",encoding='shift-jis')#文字化けするので修正
#df2['age'] = df2['age'].str.replace('～','-')

#df = pd.concat([df1,df2])
#上書き更新するので注意
#df.to_csv(data_dir /f'{output}.csv', header=True ,index=None, encoding='shift-jis')

In [87]:
df

,cp_date,year,month,card,channel,age,gender,new_account,no_of_account
0,20180331,2018,3,ゴールド,オンライン,25-29歳,女性,0,15
1,20180331,2018,3,ゴールド,オンライン,25-29歳,女性,1,1
2,20180331,2018,3,ゴールド,オンライン,25-29歳,男性,0,25
3,20180331,2018,3,ゴールド,オンライン,25-29歳,男性,1,2
4,20180331,2018,3,ゴールド,オンライン,30-34歳,女性,0,40
...,...,...,...,...,...,...,...,...,...
7432,20191231,2019,12,外商,郵送,75-79歳,男性,1,5
7433,20191231,2019,12,外商,郵送,80歳以上,女性,0,173
7434,20191231,2019,12,外商,郵送,80歳以上,女性,1,3
7435,20191231,2019,12,外商,郵送,80歳以上,男性,0,307


In [88]:
df = df[df['cp_date']>= 20180331] #2018年3月以降に限定
df

,cp_date,year,month,card,channel,age,gender,new_account,no_of_account
0,20180331,2018,3,ゴールド,オンライン,25-29歳,女性,0,15
1,20180331,2018,3,ゴールド,オンライン,25-29歳,女性,1,1
2,20180331,2018,3,ゴールド,オンライン,25-29歳,男性,0,25
3,20180331,2018,3,ゴールド,オンライン,25-29歳,男性,1,2
4,20180331,2018,3,ゴールド,オンライン,30-34歳,女性,0,40
...,...,...,...,...,...,...,...,...,...
7432,20191231,2019,12,外商,郵送,75-79歳,男性,1,5
7433,20191231,2019,12,外商,郵送,80歳以上,女性,0,173
7434,20191231,2019,12,外商,郵送,80歳以上,女性,1,3
7435,20191231,2019,12,外商,郵送,80歳以上,男性,0,307


In [89]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(7437, 9)
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            7427, 7428, 7429, 7430, 7431, 7432, 7433, 7434, 7435, 7436],
           dtype='int64', length=7437)
Index(['cp_date', 'year', 'month', 'card', 'channel', 'age', 'gender',
       'new_account', 'no_of_account'],
      dtype='object')
cp_date           int64
year              int64
month             int64
card             object
channel          object
age              object
gender           object
new_account       int64
no_of_account     int64
dtype: object


In [90]:
#日付データの変換
df['cp_date']=pd.to_datetime(df['cp_date'], format='%Y%m%d')
df.head()

,cp_date,year,month,card,channel,age,gender,new_account,no_of_account
0,2018-03-31,2018,3,ゴールド,オンライン,25-29歳,女性,0,15
1,2018-03-31,2018,3,ゴールド,オンライン,25-29歳,女性,1,1
2,2018-03-31,2018,3,ゴールド,オンライン,25-29歳,男性,0,25
3,2018-03-31,2018,3,ゴールド,オンライン,25-29歳,男性,1,2
4,2018-03-31,2018,3,ゴールド,オンライン,30-34歳,女性,0,40


In [91]:
#日付から時間を削除
df['cp_date'] = df['cp_date'].dt.strftime('%Y/%m/%d')
df.head()

,cp_date,year,month,card,channel,age,gender,new_account,no_of_account
0,2018/03/31,2018,3,ゴールド,オンライン,25-29歳,女性,0,15
1,2018/03/31,2018,3,ゴールド,オンライン,25-29歳,女性,1,1
2,2018/03/31,2018,3,ゴールド,オンライン,25-29歳,男性,0,25
3,2018/03/31,2018,3,ゴールド,オンライン,25-29歳,男性,1,2
4,2018/03/31,2018,3,ゴールド,オンライン,30-34歳,女性,0,40


In [92]:
print(df.dtypes)

cp_date          object
year              int64
month             int64
card             object
channel          object
age              object
gender           object
new_account       int64
no_of_account     int64
dtype: object


全体の入りと出を集計

In [93]:
q = """
        SELECT
        cp_date
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date
        """

df_em = sqldf(q, locals())
df_em.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,252712,4608,257320
1,2018/04/30,255557,5239,260796
2,2018/05/31,258918,6226,265144
3,2018/06/30,263343,4761,268104
4,2018/07/31,266298,4707,271005


In [94]:
#前月末を算出
q = """
        SELECT
        *
        ,LAG (eop_sum_acco, 1) OVER (ORDER BY cp_date) AS bop_sum_acco
        from 
            df_em
        group by
        cp_date
        """

df_em2 = sqldf(q, locals())
df_em2.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco,bop_sum_acco
0,2018/03/31,252712,4608,257320,NaN
1,2018/04/30,255557,5239,260796,257320.0
2,2018/05/31,258918,6226,265144,260796.0
3,2018/06/30,263343,4761,268104,265144.0
4,2018/07/31,266298,4707,271005,268104.0


In [95]:
df_em2['cancel_acco'] = df_em2['sum_new_acco'] - (df_em2['eop_sum_acco']-df_em2['bop_sum_acco'])
df_em2['cancel_rate'] = df_em2['cancel_acco']/df_em2['bop_sum_acco']
df_em2.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco,bop_sum_acco,cancel_acco,cancel_rate
0,2018/03/31,252712,4608,257320,NaN,NaN,NaN
1,2018/04/30,255557,5239,260796,257320.0,1763.0,0.006851
2,2018/05/31,258918,6226,265144,260796.0,1878.0,0.007201
3,2018/06/30,263343,4761,268104,265144.0,1801.0,0.006793
4,2018/07/31,266298,4707,271005,268104.0,1806.0,0.006736


In [96]:
#CSV出力
df_em2.to_csv(data_dir /f'{result}.csv', header=True ,index=None, encoding='shift=jis')

②有効口座数全体データの読込＆集計

In [97]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
df_cust = pd.read_csv(data_dir /f"{input_cust_all}.csv", encoding='shift-jis')
#文字化けするので修正
#df1['age'] = df1['age'].str.replace('?','～')

#df2 = pd.read_csv(data_dir /f"{input_new}.csv",encoding='shift-jis')
#df = pd.concat([df1,df2])
#上書き更新するので注意
df_cust.to_csv(data_dir /f'{output2}.csv', header=True ,index=None, encoding='shift-jis')

In [98]:
df_cust = df_cust[df_cust['cp']>= 1803] #データが2019年3月以降しか現状無し
df_cust

,cp,year,month,card,channel,age,gender,no_of_account
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,28
1,1903,2019,3,ゴールド,オンライン,25-29歳,男性,29
2,1903,2019,3,ゴールド,オンライン,30-34歳,女性,37
3,1903,2019,3,ゴールド,オンライン,30-34歳,男性,73
4,1903,2019,3,ゴールド,オンライン,35-39歳,女性,55
...,...,...,...,...,...,...,...,...
2725,2005,2020,5,外商,郵送,70-74歳,男性,14757
2726,2005,2020,5,外商,郵送,75-79歳,女性,4686
2727,2005,2020,5,外商,郵送,75-79歳,男性,10108
2728,2005,2020,5,外商,郵送,80歳以上,女性,9766


In [99]:
print(df_cust.shape)
print(df_cust.index)
print(df_cust.columns)
print(df_cust.dtypes)

(2730, 8)
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2720, 2721, 2722, 2723, 2724, 2725, 2726, 2727, 2728, 2729],
           dtype='int64', length=2730)
Index(['cp', 'year', 'month', 'card', 'channel', 'age', 'gender',
       'no_of_account'],
      dtype='object')
cp                int64
year              int64
month             int64
card             object
channel          object
age              object
gender           object
no_of_account     int64
dtype: object


In [100]:
#日付データの変換
df_cust['cp'] = df_cust['cp'] + 200000
df_cust['cp_date'] = pd.to_datetime(df_cust['cp'], format="%Y%m") + MonthEnd(1)
df_cust.head()

,cp,year,month,card,channel,age,gender,no_of_account,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,28,2019-03-31
1,201903,2019,3,ゴールド,オンライン,25-29歳,男性,29,2019-03-31
2,201903,2019,3,ゴールド,オンライン,30-34歳,女性,37,2019-03-31
3,201903,2019,3,ゴールド,オンライン,30-34歳,男性,73,2019-03-31
4,201903,2019,3,ゴールド,オンライン,35-39歳,女性,55,2019-03-31


#日付データの変換
df_cust['cp_date'] = (df_cust['cp'] + 200000)*100 + 1
df_cust['cp_date']=pd.to_datetime(df_cust['cp_date'], format='%Y%m%d')
df_cust.head()

In [101]:
#日付から時間を削除
df_cust['cp_date'] = df_cust['cp_date'].dt.strftime('%Y/%m/%d')
df_cust.head()

,cp,year,month,card,channel,age,gender,no_of_account,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,28,2019/03/31
1,201903,2019,3,ゴールド,オンライン,25-29歳,男性,29,2019/03/31
2,201903,2019,3,ゴールド,オンライン,30-34歳,女性,37,2019/03/31
3,201903,2019,3,ゴールド,オンライン,30-34歳,男性,73,2019/03/31
4,201903,2019,3,ゴールド,オンライン,35-39歳,女性,55,2019/03/31


チャネル別新規登録数

In [102]:
print(df['channel'].unique())

['オンライン' '店頭タブレット' '郵送']


In [103]:
#Web明細
q = """
        SELECT
        cp_date
        ,channel
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,channel
        """

df_cha = sqldf(q, locals())
df_cha.head()

,cp_date,channel,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,オンライン,9630,159,9789
1,2018/03/31,店頭タブレット,82142,2845,84987
2,2018/03/31,郵送,160940,1604,162544
3,2018/04/30,オンライン,9691,157,9848
4,2018/04/30,店頭タブレット,84253,3328,87581


In [104]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,channel
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,channel
        """

df_cst_cha = sqldf(q, locals())
df_cst_cha.head()

,cp_date,channel,total_cust
0,2019/03/31,オンライン,11114
1,2019/03/31,店頭タブレット,430268
2,2019/03/31,郵送,1117369
3,2019/04/30,オンライン,11171
4,2019/04/30,店頭タブレット,430348


In [105]:
df_cha2 = pd.merge(df_cha,df_cst_cha, on =['cp_date','channel'], how = 'left')
df_cha2['regist_rate'] = df_cha2['eop_sum_acco']/df_cha2['total_cust']
df_cha2

,cp_date,channel,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2018/03/31,オンライン,9630,159,9789,NaN,NaN
1,2018/03/31,店頭タブレット,82142,2845,84987,NaN,NaN
2,2018/03/31,郵送,160940,1604,162544,NaN,NaN
3,2018/04/30,オンライン,9691,157,9848,NaN,NaN
4,2018/04/30,店頭タブレット,84253,3328,87581,NaN,NaN
...,...,...,...,...,...,...,...
61,2019/11/30,店頭タブレット,127316,3307,130623,443696.0,0.294398
62,2019/11/30,郵送,179473,1588,181061,1078449.0,0.167890
63,2019/12/31,オンライン,10969,188,11157,11296.0,0.987695
64,2019/12/31,店頭タブレット,129345,2534,131879,445179.0,0.296238


In [106]:
#CSV出力
df_cha2.to_csv(data_dir /f'{result_cha}.csv', header=True ,index=None, encoding='shift=jis')

カード種類別新規登録数

In [107]:
print(df['card'].unique())

['ゴールド' '一般' '外商']


In [108]:
q = """
        SELECT
        cp_date
        ,card
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,card
        """

df_card = sqldf(q, locals())
df_card.head()

,cp_date,card,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,ゴールド,13958,294,14252
1,2018/03/31,一般,224419,3989,228408
2,2018/03/31,外商,14335,325,14660
3,2018/04/30,ゴールド,14167,326,14493
4,2018/04/30,一般,226755,4563,231318


In [109]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,card
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,card
        """

df_cst_card = sqldf(q, locals())
df_cst_card.head()

,cp_date,card,total_cust
0,2019/03/31,ゴールド,42171
1,2019/03/31,一般,1361663
2,2019/03/31,外商,154917
3,2019/04/30,ゴールド,42317
4,2019/04/30,一般,1356056


In [110]:
df_card2 = pd.merge(df_card,df_cst_card, on =['cp_date','card'], how = 'left')
df_card2['regist_rate'] = df_card2['eop_sum_acco']/df_card2['total_cust']
df_card2

,cp_date,card,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2018/03/31,ゴールド,13958,294,14252,NaN,NaN
1,2018/03/31,一般,224419,3989,228408,NaN,NaN
2,2018/03/31,外商,14335,325,14660,NaN,NaN
3,2018/04/30,ゴールド,14167,326,14493,NaN,NaN
4,2018/04/30,一般,226755,4563,231318,NaN,NaN
...,...,...,...,...,...,...,...
61,2019/11/30,一般,276227,4236,280463,1330237.0,0.210837
62,2019/11/30,外商,23554,546,24100,160640.0,0.150025
63,2019/12/31,ゴールド,18060,238,18298,42517.0,0.430369
64,2019/12/31,一般,278273,3549,281822,1327350.0,0.212319


In [111]:
#CSV出力
df_card2.to_csv(data_dir /f'{result_card}.csv', header=True ,index=None, encoding='shift=jis')

年令別

In [112]:
print(df['age'].unique())

['25-29歳' '30-34歳' '35-39歳' '40-44歳' '45-49歳' '50-54歳' '55-59歳' '60-64歳'
 '65-69歳' '70-74歳' '75-79歳' '80歳以上' '19歳以下（20歳未満）' '20-24歳']


In [113]:
q = """
        SELECT
        cp_date
        ,age
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,age
        """

df_age = sqldf(q, locals())
df_age.head()

,cp_date,age,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,19歳以下（20歳未満）,96,23,119
1,2018/03/31,20-24歳,7956,481,8437
2,2018/03/31,25-29歳,23037,610,23647
3,2018/03/31,30-34歳,29239,528,29767
4,2018/03/31,35-39歳,30992,433,31425


In [114]:
print(df_cust['age'].unique())

['25-29歳' '30-34歳' '35-39歳' '40-44歳' '45-49歳' '50-54歳' '55-59歳' '60-64歳'
 '65-69歳' '70-74歳' '75-79歳' '80歳以上' '19歳以下（20歳未満）' '20-24歳']


In [115]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,age
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,age
        """

df_cst_age = sqldf(q, locals())
df_cst_age.head()

,cp_date,age,total_cust
0,2019/03/31,19歳以下（20歳未満）,478
1,2019/03/31,20-24歳,23281
2,2019/03/31,25-29歳,52262
3,2019/03/31,30-34歳,68756
4,2019/03/31,35-39歳,84421


In [116]:
print(df_age.dtypes)

cp_date           object
age               object
sum_exist_acco     int64
sum_new_acco       int64
eop_sum_acco       int64
dtype: object


In [117]:
print(df_cst_age.dtypes)

cp_date       object
age           object
total_cust     int64
dtype: object


In [118]:
df_age2 = pd.merge(df_age,df_cst_age, on =['cp_date','age'], how = 'left')
df_age2['regist_rate'] = df_age2['eop_sum_acco']/df_age2['total_cust']
df_age2

,cp_date,age,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2018/03/31,19歳以下（20歳未満）,96,23,119,NaN,NaN
1,2018/03/31,20-24歳,7956,481,8437,NaN,NaN
2,2018/03/31,25-29歳,23037,610,23647,NaN,NaN
3,2018/03/31,30-34歳,29239,528,29767,NaN,NaN
4,2018/03/31,35-39歳,30992,433,31425,NaN,NaN
...,...,...,...,...,...,...,...
303,2019/12/31,60-64歳,25043,351,25394,160467.0,0.158251
304,2019/12/31,65-69歳,15251,187,15438,164568.0,0.093809
305,2019/12/31,70-74歳,8724,114,8838,166783.0,0.052991
306,2019/12/31,75-79歳,3511,45,3556,118740.0,0.029948


In [119]:
#CSV出力
df_age2.to_csv(data_dir /f'{result_age}.csv', header=True ,index=None, encoding='utf-8-sig')